# 想做個爬CNBC網站的爬蟲
https://www.cnbc.com/finance/

In [29]:
import sys
import pickle
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import re

In [30]:
#測試處
page_url = "https://www.cnbc.com/finance/"
r = requests.get(page_url)
r.encoding = "UTF-8"
soup = BeautifulSoup(r.text, 'html.parser')
article_node = soup.find(itemprop='articleBody')
soup.find('div', 'stories-lineup bigHeader').headline



In [35]:
#就是這一行要把href:新聞標題的內容截取下來
soup.find("div", class_="headline")


<div class="headline">
<a class=" " data-nodeid="105786243" href="/2019/03/11/brexit-is-a-disaster-thats-ruined-uk-investment-ex-barclays-chair.html">		    	                 
            Brexit an 'unmitigated disaster' that's ruined UK investment: Barclays ex-chair
			                </a> </div>

In [3]:
#複製別人的程式碼

def get_date(news_block_node):
    date_string = news_block_node.find(class_="news_date").string.split('|')[0][2:-1]
    return(datetime.strptime(date_string, '%Y.%m.%d').strftime('%Y-%m-%d'))
    
def get_title(news_block_node):
    return news_block_node.find(class_="news_title").a.string

def get_link(news_block_node):
    return news_block_node.find(class_="news_title").a.get('href')

def get_content(link):
    r = requests.get(link)
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'html.parser')
    article_node = soup.find(itemprop='articleBody')
    article = article_node.get_text()
    return article.replace("\n", "")

In [ ]:
def get_news_info(each_news):
    date  = get_date(each_news)
    title = get_title(each_news)
    link  = get_link(each_news)
    content = get_content(link)
    
    info = {'date' : date,
            'title': title,
            'link' : link,
            'content': content}
    return(info)

#主程式的架構處
def get_page_news(page_url):
    r = requests.get(page_url)
    r.encoding = "UTF-8"

    soup = BeautifulSoup(r.text, 'html.parser')
    news_blocks = soup.find_all(class_="news-list-item clearfix ")
    
    news = []
    for each_news in news_blocks:
        try:
            news_info = get_news_info(each_news)
#             print(get_title(each_news))
        except:
#             print('-------{}-------'.format())
            pass

        news.append(news_info)
    return(news)


def get_new_talk_news(from_page=1, end_page=270, url="https://www.cnbc.com/finance/"):
    print("page_number from {} to {}".format(from_page, end_page -1))
    data = []
    for page_number in range(from_page, end_page):
        print("page_number: {}".format(page_number))
        data = data + get_page_news( url+str(page_number) )
    
    print('done')
    return(data)